In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import glob
import os
import pandas as pd
import numpy as np
# from googletrans import Translator
import re

Mounted at /content/drive


In [4]:
train_word = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/train_word.npy',allow_pickle=True)

In [ ]:
len(train_word[46])

2

In [5]:
testall_word = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/testall_word.npy',allow_pickle=True)

In [165]:
testall_pos1 = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/testall_pos1.npy',allow_pickle=True)

In [166]:
testall_pos2 = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/testall_pos2.npy',allow_pickle=True)

In [167]:
testall_pos1 = testall_pos1.tolist()
testall_pos2 = testall_pos2.tolist()

In [8]:
testall_y = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/testall_y.npy',allow_pickle=True)

In [9]:
train_x = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/train_x.npy',allow_pickle=True)

In [168]:
train_y = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/train_y.npy',allow_pickle=True)

In [10]:
#position of word relative to head entity
#shape = [sample_size, sent_len]
train_pos1 = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/train_pos1.npy',allow_pickle=True)

In [11]:
train_pos2 = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/train_pos2.npy',allow_pickle=True)

In [ ]:
len(train_pos1[:50])

In [ ]:
# f = open('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/origin_data/test.txt')    
# total_words = 0
# i=0
# while True:
#   content = f.readline()
#   if content == '':
#     break
#   content = content.strip().split()
#   total_words += len(content[3])
#   i +=1
# f.close()
# print(i)
# # train_pos1 = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/train_pos1.npy',allow_pickle=True)

100


In [18]:
test_word = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/testall_word.npy',allow_pickle=True)

# Replacing 0 in train_word with 123 and 0 in temp_word with 123

In [ ]:

for i in range(len(train_word)):
  if len(train_word[i]) >=2:
    for j, item in enumerate(train_word[i]):
      if 123 in item:
        print(j, i)
  else:
    if 123 in train_word[i][0]:
      print(i)


In [ ]:
# attention_r = tf.random.normal((51,460))
# tf.reshape(attention_r, [51*460])
# sen_a = tf.random.normal((460,))
# sen_r = tf.random.normal((460,1))
# relation_embedding = tf.random.normal((12,460))
# sen_d = tf.random.normal((12,))
# gru_size=460
# attention_r[0:1,:]

In [19]:
train_word = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/train_word.npy',allow_pickle=True)

In [20]:
testall_x = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/testall_x.npy',allow_pickle=True)

In [ ]:
testall_x[0]

In [21]:
train_y = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/train_y.npy',allow_pickle=True)

In [ ]:
train_y[0]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=object)

In [ ]:
len(train_x[776])

2

In [14]:
wordembedding = np.load('/content/drive/MyDrive/Colab Notebooks/brism/relation_extraction_BiGRU_ARTT/data/vec.npy',allow_pickle=True)

In [15]:
wordembedding_add = np.append(wordembedding,np.array([[0]*100]),axis=0)

In [16]:
wordembedding_add[-1] = wordembedding_add[123]

In [17]:
wordembedding_add[123] = np.array([[123]*100])

# Replacing 123 in train_word with 16117 and 0 in train_pos1/2 with 123

In [22]:
def replace_values(list_to_replace, item_to_replace, item_to_replace_with):
    return [item_to_replace_with if item == item_to_replace else item for item in list_to_replace]

In [23]:

def replace_index(train_word,item_to_replace, item_to_replace_with):
  train_word_new = []
  for i in range(len(train_word)):
    if len(train_word[i])>=2:
      temp_list = []
      for j in train_word[i]:
        new = replace_values(j, item_to_replace, item_to_replace_with)
        temp_list.append(new)
        # print(new)
      train_word_new.append(temp_list)
    else:
      new = replace_values(train_word[i][0], item_to_replace, item_to_replace_with)
      
      train_word_new.append([new])
  return train_word_new


In [24]:
train_word_new = replace_index(train_word,123, 16117)

In [114]:
len(train_word_new[46])

2

In [117]:
def get_max_sent(train_word_new):
  set_lens = []
  for i in train_word_new:
    set_lens.append(len(i))
  return max(set_lens)

4

# Test masking


In [ ]:
x = tf.convert_to_tensor(word_lists)
# x =x  tf.Tensor(word_lists, dtype = 'int32',value_index=0)
x = tf.reshape(x, [-1,x.shape[2]])
mask = tf.not_equal(x, 16117)
mask.shape

In [ ]:
mask[46]

<tf.Tensor: shape=(70,), dtype=bool, numpy=
array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])>

In [ ]:
from tensorflow import keras

In [58]:
from keras.regularizers import Regularizer
import keras
from tensorflow.python.ops.gen_array_ops import shape
from keras.engine import training
from keras import layers
from tensorflow.python.ops.init_ops_v2 import Initializer
class GRU(object):
  def __init__(self,word_embeddings,num_steps=70,vocab_size=16118,num_classes=12,gru_size=128,
               batch_size=50,pos_num=123,pos_size=5,sent_num = 4):
    self.word_embeddings = word_embeddings
    # self.total_shape = total_shape
    self.num_steps = num_steps
    self.vocab_size = vocab_size
    self.num_classes = num_classes
    self.gru_size = gru_size
    self.sent_embed = self.gru_size*2
    self.batch_size = batch_size #batch_size = total_num(sent_set的个数),
    self.pos_num = pos_num
    self.pos_size = pos_size
    self.sent_num = sent_num 
    
  def model2(self): #输入的经过处理的flattened sentence
    #configure inputs
    # total_shape = tf.keras.layers.Input(shape=(self.batch_size+1,),dtype='int32')
    # total_shape = tf.keras.layers.Lambda(lambda x: x)(total_shape)
    # total_shape = tf.squeeze(total_shape,axis=0)
    # print(total_shape[1])
    input_word = Input(name = 'input_words', shape=(None,self.num_steps),dtype='int32') #[batch_size, sent_num, sent_len]
    input_pos1 = Input(name = 'input_pos1', shape = (None,self.num_steps),dtype='int32') #[batch_size, sent_num, sent_len]
    input_pos2 = Input(name = 'input_pos2', shape = (None,self.num_steps),dtype='int32')
    input_y = Input(name = 'input_y', shape = (self.num_classes,),dtype='float32')
    # print('input_word shape:',input_word.shape)
    input_word_reshape = tf.reshape(input_word, [-1,self.sent_num*input_word.shape[2]])
    input_pos1_reshape = tf.reshape(input_pos1, [-1,self.sent_num*input_pos1.shape[2]])
    input_pos2_reshape = tf.reshape(input_pos2, [-1,self.sent_num*input_pos2.shape[2]])
    # total_shape = total_shape[0].numpy().tolist()
    # mask = tf.not_equal(input_word_reshape, 16117)
    word_embedding = layers.Embedding(self.vocab_size,self.word_embeddings.shape[1],input_length=self.num_steps,
                                           embeddings_initializer=tf.initializers.Constant(self.word_embeddings),
                                      trainable=False,name = 'word_embedding_matrix')
    
    pos1_embedding = layers.Embedding(self.pos_num,self.pos_size,name = 'pos1_embedding_matrix') 
    pos2_embedding = layers.Embedding(self.pos_num,self.pos_size,name = 'pos2_embedding_matrix') 
    input = word_embedding(input_word_reshape) 
    pos1 = pos1_embedding(input_pos1_reshape) 
    pos2 = pos2_embedding(input_pos2_reshape)

    input = tf.reshape(input, [-1,self.num_steps,self.word_embeddings.shape[1]]) #[batch_size*sen_num, num_step, embed_size]
    pos1 = tf.reshape(pos1, [-1,self.num_steps,self.pos_size])
    pos2 = tf.reshape(pos2, [-1,self.num_steps,self.pos_size])
   
    x = layers.Concatenate(axis=-1)([input,pos1,pos2]) ##[batch_size*sen_num, num_step, embed_size(3)]
    masking_layer= layers.Masking(mask_value=123,
                                  input_shape=(self.batch_size*self.sent_num, self.num_steps), name= 'masking_layer')(x)
    # x = masking_layer(x)
    # print(x._keras_mask)
    # print(masking_layer[0])
    self.gru_layer = layers.Bidirectional(layers.GRU(self.gru_size,return_sequences=True,
                                                kernel_regularizer=tf.keras.regularizers.L2(0.001)))(masking_layer) #[batch_size*sen_num, num_step, gru_size*2]
    dropout_layer= layers.Dropout(0.5)(self.gru_layer)   
    self.dropout_layer = tf.reshape(dropout_layer, [-1, self.sent_num,self.num_steps,self.gru_size*2])     
    self.word_attention_layer = Attention1(self.batch_size,self.num_steps,self.sent_num)(self.dropout_layer)
    print("Mask found:", self.gru_layer._keras_mask)
    # print('word_attention_layer shape:',word_attention_layer.shape) #[flattened_sentence_num,gru_size*2]
    outputs = Attention2(self.sent_embed,self.batch_size,self.sent_num,self.num_classes)(self.word_attention_layer)
    
    # 
    model = keras.Model(inputs = [input_word,input_pos1,input_pos2], outputs = outputs)
    return model
class Attention1(Layer): 
  def __init__(self,batch_size,num_steps,sent_num):
    super(Attention1, self).__init__()
    self.batch_size = batch_size
    self.num_steps = num_steps
    self.sent_num = sent_num
    self.supports_masking = True

  def build(self,input_shape):
    #相当于weight initalization
    self.attention_w = self.add_weight(name = 'attention_omega', shape = (input_shape[-1],1), initializer="random_normal",
                                       trainable=True,regularizer=tf.keras.regularizers.L2(0.001)) #W shape[gru_size,1]
    #attention_w用来GRUlayer输出的sentence_embedding（hidden representation of sentences)，提取word_level对于sentence relation
    #有重要意义的word，得到的sentence-level feature 
    # self.sen_a = self.add_weight(name = 'attention_A', shape=(gru_size),trainable=True)
    #以上两个用在sentence_set level, 用attention_A提取对于预测sentence的relation有帮助的同样隶属于这个relation的游泳的sentence instances
    super(Attention1,self).build(input_shape)
  def call(self, x):
    # x[batch_size,sen_num, num_step, gru_size*2]
    total_sent = self.batch_size*self.sent_num
    sent_embed = x.shape[3]
    x_reshape = tf.reshape(x, [total_sent, self.num_steps,sent_embed])
    # print('x_reshape',x_reshape[0],x_reshape[1],x_reshape[3])
    M = tf.reshape(tf.tanh(x), [total_sent*self.num_steps,sent_embed])
    
    alpha = tf.matmul(M, self.attention_w)
    #[total_sent*num_step,gru_size*2][gru_size*2,1] = [total_sent*num_step,1]
    alpha = tf.reshape(alpha,[total_sent,self.num_steps]) #[total_sent*num_step,1] - > [total_sent, num_steps]
    # print('alpha',alpha[0],alpha[1],alpha[3])
    alpha = tf.nn.softmax(alpha,axis=-1) #[total_sent, num_steps]
    alpha = tf.reshape(alpha,[total_sent,1,self.num_steps])#[total_sent, num_steps] ->[total_sent,1,num_step] 
    r = tf.matmul(alpha, x_reshape)#[total_sent,1,num_step][total_sent, num_step,gru_size] = [total_sent,1,gru_size]
    sent_embedding = tf.reshape(r,[self.batch_size,self.sent_num,sent_embed]) #[total_sent,1,gru_size*2]-> [batch_size,sen_num,gru_size*2]
    
    return sent_embedding

#sentence-level attention
class Attention2(Layer): 
  def __init__(self,sent_embed,batch_size,sent_num,num_class):
    super(Attention2, self).__init__()
    self.sent_embed =sent_embed
    self.batch_size = batch_size
    self.num_class = num_class
    self.sent_num = sent_num
    self.supports_masking = True
  
  def build(self,input_shape):
    self.sen_a = self.add_weight(name = 'attention_A', shape=(input_shape[-1],),trainable=True,initializer="random_normal",
                                       regularizer=tf.keras.regularizers.L2(0.001))
                                 
                                #  regularizer=tf.keras.regularizers.L2(0.001))
    self.sen_r =self.add_weight(name = 'query_r', shape=(input_shape[-1],1),trainable=True,initializer="random_normal",
                                       regularizer=tf.keras.regularizers.L2(0.001))
                                # regularizer=tf.keras.regularizers.L2(0.001))
    self.relation_embedding = self.add_weight(name = 'relation_embedding', shape=(self.num_class,input_shape[-1]),trainable=True,initializer="random_normal",
                                       regularizer=tf.keras.regularizers.L2(0.001))
                                              # regularizer=tf.keras.regularizers.L2(0.001))
    self.sen_d = self.add_weight(name = 'bias', shape=(self.num_class,),initializer="zeros",trainable=True,
                                       regularizer=tf.keras.regularizers.L2(0.001))
                                #  regularizer=tf.keras.regularizers.L2(0.001))
    super(Attention2,self).build(input_shape)
  def call(self,x):

    #x[batch_size,sen_num,gru_size*2]
    sent_repre = [] #list of sentence-set,#每个sent_set将其对应的所有句子的表达提取出来，放进sent_repre list
    sent_alpha = [] #list of set-level attention weights
    set_repre =[]
    relation_score =[]
    relation_prob =[]
    relation_prediction = []
    loss = []
    for i in range(self.batch_size): #loop through sent_sets within one batch
      # for j in range(self.sent_num):
      set_sent = x[i] #[sen_num,gru_size*2]
      sent_repre.append(tf.tanh(set_sent))
      e = tf.multiply(set_sent,self.sen_a) #[sen_num, gru_size*2][gru_size*2,] = [sen_num, gru_size*2](element-wise multiplication) 
      e = tf.matmul(e,self.sen_r) #[sen_num, gru_size*2][gru_size*2,1] = [sen_num,1] 
      e_reshape = tf.reshape(e, [self.sent_num]) #[sen_num,1] -> [sen_num]
      set_prob = tf.nn.softmax(e_reshape)  #[sen_num] -> [sen_num]
      set_prob_reshape = tf.reshape(set_prob, [1,self.sent_num]) #[sen_num]-> [1,sen_num]
      sent_alpha.append(set_prob_reshape)
      
      s = tf.matmul(sent_alpha[i],sent_repre[i]) #[1,sen_num][sen_num,gru_size*2] = [1,gru_size*2] #整个set的表达（包含sent的不用的重要程度的信息）
      #每个句子有不同的weight，#wegihted sum->得到代表着一个set的表达，表示不同的句子/relation对于表达这个set的贡献不同，
      #反过来说，这个set包含的relation们的主导程度
      #纬度跟单个句子的纬度相同
      s_reshape = tf.reshape(s, [self.sent_embed,1])
      set_repre.append(s_reshape)
      
      o=tf.matmul(self.relation_embedding, set_repre[i]) #[num_class, gru_size*2][gru_size*2,1] = [num_class,1]
      #这个set属于不同relation的可能性
      o_reshape = tf.reshape(o, [self.num_class]) #[num_class,1] -> [num_class]
      # print(o_reshape, self.sen_d.shape)
      o = tf.add(o_reshape,self.sen_d)
      relation_score.append(o) #[batch_size,num_class]
      relation_prob.append(tf.nn.softmax(relation_score[i]))
      relation_prediction.append(tf.argmax(relation_prob[i],0))
    # print('sent_alpha',sent_alpha[0],sent_alpha[1])
    return relation_score,relation_prob,relation_prediction
  

In [59]:
gru = GRU(wordembedding_add,num_steps=70,vocab_size=16118,num_classes=12,gru_size=128,
               batch_size=32,pos_num=124,pos_size=6,sent_num=4)
model = gru.model2()


Mask found: KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.bool, name=None), name='Placeholder_2:0')


In [ ]:
output = model([np.array(word_lists), np.array(pos1_lists), np.array(pos2_lists)])

In [ ]:
output[1][0]

<tf.Tensor: shape=(12,), dtype=float32, numpy=
array([0.11349161, 0.09907483, 0.08728728, 0.08673273, 0.0657768 ,
       0.08569241, 0.06923382, 0.10147817, 0.07296032, 0.06079897,
       0.06248864, 0.09498443], dtype=float32)>

In [ ]:
output[2]

In [ ]:
output[1][0]
tf.argmax(output[1][46],0)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [ ]:
# class TemporalSoftmax(keras.layers.Layer):
#     def call(self, inputs, mask=None):
#         broadcast_float_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
#         inputs_exp = tf.exp(inputs) * broadcast_float_mask
#         inputs_sum = tf.reduce_sum(
#             inputs_exp * broadcast_float_mask, axis=-1, keepdims=True
#         )
#         print(mask)
#         return inputs_exp / inputs_sum


# inputs = keras.Input(shape=(None,), dtype="int32")
# x = layers.Embedding(input_dim=10, output_dim=32, mask_zero=True)(inputs)
# x = layers.Dense(1)(x)
# outputs = TemporalSoftmax()(x)

# model = keras.Model(inputs, outputs)
# y = model(np.random.randint(0, 10, size=(32, 100)), np.random.random((32, 100, 1)))

Tensor("Placeholder_1:0", shape=(None, None), dtype=bool)
tf.Tensor(
[[ True  True  True ...  True  True  True]
 [ True False  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True False  True ...  True  True  True]
 [ True  True  True ...  True  True  True]], shape=(32, 100), dtype=bool)


In [ ]:
model._

In [ ]:
output[-1]

In [ ]:
# samples, timesteps, features = 32, 10, 8
# inputs = np.random.random([samples, timesteps, features]).astype(np.float32)
# inputs[:, 3, :] = 0.
# inputs[:, 5, :] = 0.

# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Masking(mask_value=0.,
#                                   input_shape=(timesteps, features)))
# model.add(tf.keras.layers.LSTM(32))

# output = model(inputs)

In [ ]:
# class CustomEmbedding(keras.layers.Layer):
#     def __init__(self, input_dim, output_dim, mask_zero=False, **kwargs):
#         super(CustomEmbedding, self).__init__(**kwargs)
#         self.input_dim = input_dim
#         self.output_dim = output_dim
#         self.mask_zero = mask_zero

#     def build(self, input_shape):
#         self.embeddings = self.add_weight(
#             shape=(self.input_dim, self.output_dim),
#             initializer="random_normal",
#             dtype="float32",
#         )

#     def call(self, inputs):
#         return tf.nn.embedding_lookup(self.embeddings, inputs)

#     def compute_mask(self, inputs, mask=None):
#         if not self.mask_zero:
#             return None
#         return tf.not_equal(inputs, 123)


# layer = CustomEmbedding(124, 32, mask_zero=True)
# x = tf.convert_to_tensor(news, dtype= tf.int32)
# y = layer(x)
# mask = layer.compute_mask(x)

# print(mask)

tf.Tensor(
[[ True  True  True  True False  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True False]
 [ True  True  True  True False  True  True  True  True  True]], shape=(3, 10), dtype=bool)


In [ ]:
# def calculate_loss(total_num,loss_fn,y):
#   for i in total_num:
#     loss = loss_fn()

In [ ]:
# def custom_loss_function(y_true, y_pred):
#    squared_difference = tf.square(y_true - y_pred)
#    return tf.reduce_mean(squared_difference, axis=-1)

# model.compile(optimizer='adam', loss=custom_loss_function)

In [ ]:
# import numpy as np

# y_true = [12, 20, 29., 60.]
# y_pred = [14., 18., 27., 55.]
# cl = custom_loss_function(np.array(y_true),np.array(y_pred))
# cl.numpy()

9.25

In [ ]:
# optimizer = tf.keras.optimizers.Adam()
# optimizer.apply_gradients

<bound method OptimizerV2.apply_gradients of <keras.optimizer_v2.adam.Adam object at 0x7f8ab67fb390>>

In [133]:
len(tem_pos1[1]), len(temp_word[1])

(1, 1)

In [ ]:
[tem_pos1[0]]+[[123]*70]*(4-len(temp_word[0]))

In [121]:
temp_order_train = list(range(len(test_word_new)))
np.random.shuffle(temp_order_train)
data = []
batch_size =32
i = 0
# for i in range(int(len(temp_order_train) / batch_size)):
temp_input = temp_order_train[i * batch_size:(i+1) * batch_size]
temp_word = []
tem_pos1 = []
tem_pos2 = []
temp_y = []
for k in temp_input:
  temp_word.append(train_word[k])
  tem_pos1.append(testall_pos1[k])
  tem_pos2.append(testall_pos2[k])
  temp_y.append(testall_y[k])
total_word = [] #total sent (flatten sent_set)
total_pos1 = []
total_pos2 = []
# for i in range(len(temp_word)):
#     total_num.append(len(temp_word[i]))
word_lists =[]
pos1_lists = []
pos2_lists = []
for id in range(len(temp_word)):
  if len(temp_word[id])< 4:
    word_lists.append(temp_word[id]+[[123]*70]*(4-len(temp_word[id])))
    pos1_lists.append(tem_pos1[id]+[[123]*70]*(4-len(temp_word[id])))
    print(pos1_lists)
    pos2_lists.append(tem_pos2[id]+[[123]*70]*(4-len(temp_word[id])))
  else:
    word_lists.append(temp_word[id])
    pos1_lists.append(tem_pos1[id])
    pos2_lists.append(tem_pos2[id])

In [ ]:
tem_pos1[0].tolist()+[[123]*70]*(4-len(temp_word[0]))

In [178]:
batch_size =32
def process_data(train_word,train_pos1,train_pos2,train_y): 
  temp_order_train = list(range(len(train_word)))
  np.random.shuffle(temp_order_train)
  data = []
  for i in range(int(len(temp_order_train) / batch_size)):
    temp_input = temp_order_train[i * batch_size:(i+1) * batch_size]
    temp_word = []
    tem_pos1 = []
    tem_pos2 = []
    temp_y = []
    for k in temp_input:
      temp_word.append(train_word[k])
      tem_pos1.append(train_pos1[k])
      tem_pos2.append(train_pos2[k])
      temp_y.append(train_y[k])
    # total_shape = [] #第i个元素表示前i个sent_set一共多少个sentence
    # total_num = []
    total_word = [] #total sent (flatten sent_set)
    total_pos1 = []
    total_pos2 = []
    # for i in range(len(temp_word)):
    #     total_num.append(len(temp_word[i]))
    word_lists =[]
    pos1_lists = []
    pos2_lists = []
    for id in range(len(temp_word)):
      if len(temp_word[id])< 4:
        word_lists.append(temp_word[id]+[[123]*70]*(4-len(temp_word[id])))
        pos1_lists.append(tem_pos1[id]+[[123]*70]*(4-len(temp_word[id])))
        # print(pos1_lists)
        pos2_lists.append(tem_pos2[id]+[[123]*70]*(4-len(temp_word[id])))
      else:
        word_lists.append(temp_word[id])
        pos1_lists.append(tem_pos1[id])
        pos2_lists.append(tem_pos2[id])
    # break
    
    word_batch = tf.convert_to_tensor(word_lists)
    pos1_batch = tf.convert_to_tensor(pos1_lists)
    pos2_batch = tf.convert_to_tensor(pos2_lists)
    y_batch = tf.convert_to_tensor(temp_y)
    data.append((word_batch,pos1_batch,pos2_batch,y_batch))
  
  return data

In [ ]:
data = process_data(train_word_new,train_pos1,train_pos2,train_y)

In [ ]:
testall_pos1[0]

In [179]:
data = process_data(test_word_new,testall_pos1,testall_pos2,testall_y)

## Training with fit from scratch

In [204]:
test_word_new = replace_index(testall_word,123, 16117)
train_word_new = replace_index(train_word,123, 16117)
batch_size=32
epochs = 20
# loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# optimizer = tf.keras.optimizers.Adam()
# acc_metric = keras.metrics.Accuracy()
class Training(keras.Model):
  def __init__(self,model, batch_size=32, epochs=20,num_class=12,sent_num = 4):
    super(Training,self).__init__()
    self.model = model
    self.batch_size = batch_size
    self.epochs = epochs
    self.num_class = num_class
    self.sent_num =sent_num

  def get_databatch(self,train_word,train_pos1,train_pos2,train_y,testall_word,testall_pos1,testall_pos2,testall_y):
    self.train_dataset = self.process_data(train_word,train_pos1,train_pos2,train_y)
    self.test_dataset = self.process_data(testall_word,testall_pos1,testall_pos2,testall_y)
  def process_data(self, train_word,train_pos1,train_pos2,train_y): 
    temp_order = list(range(len(train_word)))
    np.random.shuffle(temp_order)
    data = []
    for i in range(int(len(temp_order) / self.batch_size)):
      temp_input = temp_order[i * self.batch_size:(i+1) * self.batch_size]
      temp_word = []
      tem_pos1 = []
      tem_pos2 = []
      temp_y = []
      for k in temp_input:
        temp_word.append(train_word[k])
        tem_pos1.append(train_pos1[k])
        tem_pos2.append(train_pos2[k])
        temp_y.append(train_y[k])
      # total_shape = [] #第i个元素表示前i个sent_set一共多少个sentence
      # total_num = []
      total_word = [] #total sent (flatten sent_set)
      total_pos1 = []
      total_pos2 = []
      # for i in range(len(temp_word)):
      #     total_num.append(len(temp_word[i]))
      word_lists =[]
      pos1_lists = []
      pos2_lists = []
      for id in range(len(temp_word)):
        if len(temp_word[id])< self.sent_num:
          word_lists.append(temp_word[id]+[[123]*70]*(self.sent_num-len(temp_word[id])))
          pos1_lists.append(tem_pos1[id]+[[123]*70]*(self.sent_num-len(temp_word[id])))
          pos2_lists.append(tem_pos2[id]+[[123]*70]*(self.sent_num-len(temp_word[id])))
        else:
          word_lists.append(temp_word[id])
          pos1_lists.append(tem_pos1[id])
          pos2_lists.append(tem_pos2[id])
      
      word_batch = tf.convert_to_tensor(word_lists)
      pos1_batch = tf.convert_to_tensor(pos1_lists)
      pos2_batch = tf.convert_to_tensor(pos2_lists)
      y_batch = tf.convert_to_tensor(temp_y)
      data.append((word_batch,pos1_batch,pos2_batch,y_batch))
    
    return data
  

  def train(self):
    for one_epoch in range(self.epochs): 
      print('Epoch {}'.format(one_epoch) +'......')
      for i, one_batch_train in enumerate(self.train_dataset):
        print('   Training batch {}'.format(i) +':' )
        # print('y_batch shape',y_batch.shape)
        self.train_step(one_batch_train)
      
      for j, one_batch_test in enumerate(self.test_dataset):
        print('   Test batch {}'.format(j) +':' )
        self.test_step(one_batch_test)
  def train_step(self, one_batch):
    x = one_batch[:3]
    y= one_batch[3]
    loss_tracker = keras.metrics.CategoricalCrossentropy(name="loss")
    accuracy_tracker = keras.metrics.Accuracy(name='accuracy')
    loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.Adam()
    # # Prepare the metrics.
    # train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    # val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    # Iterate over the batches of a dataset.
    with tf.GradientTape() as tape:

      outputs = self.model(x, training=True) #输入的都是flattened set sentences
      logits = tf.convert_to_tensor(outputs[0])
      # print('y_array :',y_array,'logits:',logits)
      loss = loss_fn(y,logits) 
      # print('loss:',loss)
    preditions= [o.numpy() for o in outputs[2]]
    y_pred = tf.one_hot(preditions, self.num_class)
    y = y.numpy()
    prob = [i.numpy().tolist() for i in outputs[1]]
    # print('prob:',prob,'y:',y)
    #compute gradients
    trainable_vars =self.model.trainable_variables
    gradients=tape.gradient(loss,trainable_vars)
    #update weights
    optimizer.apply_gradients(zip(gradients, trainable_vars))
    #update metric
    # print('loss_tracker1:',loss_tracker)
    loss_tracker.update_state(y,prob)
    # print('loss_tracker2:',loss_tracker.result())
    accuracy_tracker.update_state(y,y_pred)
    print({"loss": loss_tracker.result().numpy(), "accuracy": accuracy_tracker.result().numpy()})
  def test_step(self, one_batch):
    x = one_batch[:3]
    y= one_batch[3]
    accuracy_tracker = keras.metrics.Accuracy(name='accuracy')
    # with tf.GradientTape() as tape:
    outputs = self.model(x, training=False) #输入的都是flattened set sentences
      # loss = loss_fn(y_array,logits)
    preditions=  [o.numpy() for o in outputs[2]]
    y_pred = tf.one_hot(preditions, self.num_class)
    y = y.numpy()
    accuracy_tracker.update_state(y_pred,y)
    print({"accuracy": accuracy_tracker.result().numpy()})



In [205]:
training = Training(model,batch_size=32, epochs=20,num_class =12,sent_num = 4)

In [206]:
training.get_databatch(train_word_new,train_pos1,train_pos2,train_y,
                       test_word_new,testall_pos1,testall_pos2,testall_y)

In [207]:
training.train()

Epoch 0......
   Training batch 0:
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/keras/backend.py", line 4705, in <genexpr>
    for ta, out in zip(output_ta_t, flat_new_output))  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/keras/backend.py", line 4705, in <genexpr>
    for ta, out in zip(output_ta_t, flat_new_output))  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/keras/backend.py", line 4705, in <genexpr>
    for ta, out in zip(output_ta_t, flat_new_output))  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/keras/backend.py", line 4705, in <genexpr>
    for ta, out in zip(output_ta_t, flat_new_output))  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


{'loss': 1.5195329, 'accuracy': 0.921875}
   Training batch 1:
{'loss': 1.7633479, 'accuracy': 0.90625}
   Training batch 2:
{'loss': 1.5448565, 'accuracy': 0.9166667}
   Training batch 3:
{'loss': 1.6985173, 'accuracy': 0.9270833}
   Training batch 4:
{'loss': 1.8634574, 'accuracy': 0.9010417}
   Training batch 5:
{'loss': 1.6561835, 'accuracy': 0.9114583}
   Training batch 6:
{'loss': 1.6949928, 'accuracy': 0.921875}
   Training batch 7:
{'loss': 2.0039253, 'accuracy': 0.875}
   Training batch 8:
{'loss': 1.2872593, 'accuracy': 0.9270833}
   Training batch 9:
{'loss': 1.646263, 'accuracy': 0.921875}
   Training batch 10:
{'loss': 1.7782829, 'accuracy': 0.9010417}
   Training batch 11:
{'loss': 1.7266119, 'accuracy': 0.9114583}
   Training batch 12:
{'loss': 1.833074, 'accuracy': 0.9114583}
   Training batch 13:
{'loss': 1.8117883, 'accuracy': 0.921875}
   Training batch 14:
{'loss': 2.3699613, 'accuracy': 0.8802083}
   Training batch 15:
{'loss': 1.859924, 'accuracy': 0.9010417}
   T

In [199]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

In [201]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                             monitor='accuracy',
                             verbose=1,
                             save_best_only=True, 
                             save_weights_only=True, 
                             mode='max')

In [203]:
os.listdir(checkpoint_dir)

FileNotFoundError: ignored

In [202]:
checkpoint

In [ ]:
def main():
    # the path to save models
    save_path = './model/'

    print('reading wordembedding')
    wordembedding = np.load('./data/vec.npy')

    print('reading training data')
    train_y = np.load('./data/train_y.npy')
    train_word = np.load('./data/train_word.npy')
    train_pos1 = np.load('./data/train_pos1.npy')
    train_pos2 = np.load('./data/train_pos2.npy')

    vocab_size = len(wordembedding)
    num_classes = len(train_y[0])
    total_num = 50

    def train_step(word_batch, pos1_batch, pos2_batch, y_batch, total_num):
      
